<h1> Part One: Scraping</h1>

In [7]:
#install beautifule soup
#!pip install beautifulsoup4

In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
import urllib.request

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
page = urllib.request.urlopen(url)

In [6]:
from bs4 import BeautifulSoup

In [7]:
soup = BeautifulSoup(page, "lxml")

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNames

In [9]:
#find the table
right_table = soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [10]:
#format the data into columns for the dataframe
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)>0:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [11]:
#add data to the dataframe and assign column names that I didn't scrape
import pandas as pd
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postal Code     288 non-null object
Borough         288 non-null object
Neighborhood    288 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


In [13]:
#Drop rows where Borough is not assigned
df2 = df[df['Borough'] != 'Not assigned'] 
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 2 to 286
Data columns (total 3 columns):
Postal Code     211 non-null object
Borough         211 non-null object
Neighborhood    211 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


In [14]:
#view the emaining rows where neighborhood is not assigned (I'm also stripping some whitespace out)
df2[df2['Neighborhood'].map(lambda x: x.strip()) == "Not assigned"]

,Postal Code,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [15]:
#default those rows to use borough & check the results
import numpy as np
df2['Neighborhood'] = np.where(df2['Neighborhood'].map(lambda x: x.strip()) == "Not assigned", df2['Borough'], df2['Neighborhood'])
df2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [16]:
#group the postcodes with more than one neighborhood into one line
df3 = df2.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(' , '.join).reset_index()
df3

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood\n , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West\n"


In [17]:
#remove the newline characters
df4 = df3.replace(r'\n',' ', regex=True)
df4

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [18]:
#final shape
df4.shape

(103, 3)

<h1>Part Two - Get the Lat/Lon Coordinates</h1>

In [19]:
#the geocoder wasn't returning so I decided to use the csv file
geo = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
#merge the two dataframes 
df5 = pd.merge(df4, geo, on='Postal Code')
df5

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


<h1>Part Three - The actual clustering! </h1>

In [22]:
#getting the all libraries we'll need 

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

<h2>Explore Toronto and select a borough to analyze more deeply</h2>

In [35]:
#define a user agent and get the coordinates for the center of Toronto using geopy
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.


In [36]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
#limit to just the scarborough borough so we can analyze this borough more deeply
scarborough_data = df5[df5['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
#get the lat/lon of the center of scarborough borough
address = 'Scarborough, Canada'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [39]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

<h2>Start looking at venue information for the selected borough using Foursquare</h2>

In [40]:
#SENSITIVE
#pass in Foursquare developer credentials
CLIENT_ID = 'redacted' # your Foursquare ID
CLIENT_SECRET = 'redacted' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [41]:
#look at a one neighborhood in scarborough
scarborough_data.loc[3, 'Neighborhood']

'Woburn'

In [42]:
#get the lat and lon for that one neighborhood
neighborhood_latitude = scarborough_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


In [43]:
#get the top 100 venus in a 500 meter radius of Woburn
LIMIT = 100
radius = 500 # define radius 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=22PBJLNJF2XWNCXYUODV1EXTYUA3CAVOECK01H1AVSWE2T0N&client_secret=DADRJ1SBCPLEGFZPE1YYY4BUJSCM0RL0E0YDDFTSIDRFMEYD&v=20180605&ll=43.7709921,-79.21691740000001&radius=500&limit=100'

In [44]:
#show the results as a json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db210913bf999001b9c4729'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7754921045, 'lng': -79.21069729639068},
   'sw': {'lat': 43.7664920955, 'lng': -79.22313750360935}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cc1d28c06c254815ac18547',
       'name': 'Starbucks',
       'location': {'address': '300 Borough Dr',
        'crossStreet': 'Scarborough Town Centre',
        'lat': 43.770037201625215,
        'lng': -79.22115586641958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.770037201625215,
          'lng': -79.22115586641958}],
        'distance': 356,
        'cc': 'CA

In [50]:
#create a function to get nearby venues for all of the neighborhoods in scarborough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
#run the function
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge , Malvern
Highland Creek , Rouge Hill , Port Union
Guildwood  , Morningside , West Hill
Woburn
Cedarbrae 
Scarborough Village
East Birchmount Park  , Ionview , Kennedy Park
Clairlea , Golden Mile , Oakridge
Cliffcrest , Cliffside , Scarborough Village West 
Birch Cliff , Cliffside West 
Dorset Park , Scarborough Town Centre , Wexford Heights
Maryvale , Wexford
Agincourt
Clarks Corners  , Sullivan  , Tam O'Shanter
Agincourt North , L'Amoreaux East  , Milliken , Steeles East 
L'Amoreaux West 
Upper Rouge


In [52]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge , Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [53]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
"Agincourt North , L'Amoreaux East , Milliken , Steeles East",2,2,2,2,2,2
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea , Golden Mile , Oakridge",10,10,10,10,10,10
"Clarks Corners , Sullivan , Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest , Cliffside , Scarborough Village West",2,2,2,2,2,2
"Dorset Park , Scarborough Town Centre , Wexford Heights",8,8,8,8,8,8
"East Birchmount Park , Ionview , Kennedy Park",6,6,6,6,6,6


In [54]:
print('There are {} unique categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 unique categories.


In [55]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge , Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek , Rouge Hill , Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
scarborough_onehot.shape

(89, 54)

In [57]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000,0.0,0.0,0.00,0.000000,0.166667,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000,0.166667,0.000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.166667,0.000000,0.166667,0.00,0.000000,0.0,0.000000,0.000,0.000
1,"Agincourt North , L'Amoreaux East , Milliken ...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.5,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000,0.000
2,"Birch Cliff , Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000,0.000
3,Cedarbrae,0.0,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.000,0.0,0.0,0.00,0.142857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.00,0.000000,0.142857,0.000000,0.00,0.000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.142857,0.000,0.000
4,"Clairlea , Golden Mile , Oakridge",0.0,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.000,0.2,0.1,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.100,0.000000,0.00,0.000,0.000000,0.000,0.1,0.000,0.000000,0.0,0.000,0.000000,0.1,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.1,0.000000,0.000,0.000
5,"Clarks Corners , Sullivan , Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.000,0.0,0.0,0.00,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.090909,0.090909,0.00,0.000000,0.000000,0.000000,0.00,0.000,0.090909,0.00,0.000,0.000000,0.000,0.0,0.000,0.000000,0.0,0.000,0.090909,0.0,0.000,0.090909,0.181818,0.000000,0.000000,0.090909,0.000000,0.00,0.000000,0.0,0.090909,0.000,0.000
6,"Cliffcrest , Cliffside , Scarborough Village W...",0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000,0.000000,0.000,0.0,0.000,0.000000,0.5,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000,0.000
7,"Dorset Park , Scarborough Town Centre , Wexfor...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125,0.0,0.0,0.00,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.00,0.125,0.000

In [59]:
scarborough_grouped.shape

(16, 54)

In [61]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge  0.17
1      Breakfast Spot  0.17
2      Sandwich Place  0.17
3          Print Shop  0.17
4  Chinese Restaurant  0.17


----Agincourt North , L'Amoreaux East  , Milliken , Steeles East ----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff , Cliffside West ----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae ----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea , Golden Mile , Oakridge----
          venue  freq
0        Bakery   0.2
1      Bus Line   0.2
2      

In [109]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Sandwich Place,Print Shop,Breakfast Spot,Lounge,Chinese Restaurant,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North , L'Amoreaux East , Milliken ...",Park,Playground,Caribbean Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Convenience Store
2,"Birch Cliff , Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Caribbean Restaurant,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Auto Garage,Grocery Store,General Entertainment
4,"Clairlea , Golden Mile , Oakridge",Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Coffee Shop


<h2>Cluster the neighborhoods</h2>

In [127]:
# set number of clusters
kclusters = 7

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 5, 2, 2, 2, 2, 4, 2, 0, 2], dtype=int32)

In [128]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353,6.0,Fast Food Restaurant,Vietnamese Restaurant,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497,1.0,Bar,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,2.0,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Moving Target,Vietnamese Restaurant,Clothing Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Caribbean Restaurant,Bakery,Fried Chicken Joint,Bank,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Auto Garage,Grocery Store,General Entertainment
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,5.0,Grocery Store,Playground,Convenience Store,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029,0.0,Chinese Restaurant,Hobby Shop,Discount Store,Department Store,Convenience Store,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577,2.0,Bakery,Bus Line,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Coffee Shop
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W...",43.716316,-79.239476,4.0,American Restaurant,Motel,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,2.0,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [129]:
scarborough_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 16 columns):
Postal Code               17 non-null object
Borough                   17 non-null object
Neighborhood              17 non-null object
Latitude                  17 non-null float64
Longitude                 17 non-null float64
Cluster Labels            16 non-null float64
1st Most Common Venue     16 non-null object
2nd Most Common Venue     16 non-null object
3rd Most Common Venue     16 non-null object
4th Most Common Venue     16 non-null object
5th Most Common Venue     16 non-null object
6th Most Common Venue     16 non-null object
7th Most Common Venue     16 non-null object
8th Most Common Venue     16 non-null object
9th Most Common Venue     16 non-null object
10th Most Common Venue    16 non-null object
dtypes: float64(3), object(13)
memory usage: 2.2+ KB


In [130]:
#drop Upper Rouge Neighborhood because it did not return any venues and check to make sure its dropped
scarborough_merged = scarborough_merged.dropna()
scarborough_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 16 columns):
Postal Code               16 non-null object
Borough                   16 non-null object
Neighborhood              16 non-null object
Latitude                  16 non-null float64
Longitude                 16 non-null float64
Cluster Labels            16 non-null float64
1st Most Common Venue     16 non-null object
2nd Most Common Venue     16 non-null object
3rd Most Common Venue     16 non-null object
4th Most Common Venue     16 non-null object
5th Most Common Venue     16 non-null object
6th Most Common Venue     16 non-null object
7th Most Common Venue     16 non-null object
8th Most Common Venue     16 non-null object
9th Most Common Venue     16 non-null object
10th Most Common Venue    16 non-null object
dtypes: float64(3), object(13)
memory usage: 2.1+ KB


In [131]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters